In [1]:
#import this before other imports
from sklearnex import patch_sklearn,unpatch_sklearn

patch_sklearn()
#unpatch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
from tld import get_tld
import re
from sklearn.model_selection import train_test_split
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score,log_loss,precision_score,recall_score,make_scorer,accuracy_score
import json

In [2]:
l1 = pd.read_csv("FinalDataset/URL/Benign_list_big_final.csv", header=None)
l1

,0
0,http://1337x.to/torrent/1048648/American-Snipe...
1,http://1337x.to/torrent/1110018/Blackhat-2015-...
2,http://1337x.to/torrent/1122940/Blackhat-2015-...
3,http://1337x.to/torrent/1124395/Fast-and-Furio...
4,http://1337x.to/torrent/1145504/Avengers-Age-o...
...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...
35374,https://lastpass.com/signup2.php?ac=1&from_uri...
35375,https://lastpass.com/signup2.php?ac=1&from_uri...
35376,https://lastpass.com/signup2.php?ac=1&from_uri...


In [3]:
l1['result'] = [0 for _ in range(l1.shape[0])]
l1

,0,result
0,http://1337x.to/torrent/1048648/American-Snipe...,0
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0
...,...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35374,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35375,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35376,https://lastpass.com/signup2.php?ac=1&from_uri...,0


In [4]:
l2 = pd.read_csv("FinalDataset/URL/DefacementSitesURLFiltered.csv", header=None)
l2['result'] = [1 for _ in range(l2.shape[0])]
l3 = pd.read_csv("FinalDataset/URL/Malware_dataset.csv", header=None)
l3['result'] = [1 for _ in range(l3.shape[0])]
l4 = pd.read_csv("FinalDataset/URL/phishing_dataset.csv", header=None)
l4['result'] = [1 for _ in range(l4.shape[0])]
l5 = pd.read_csv("FinalDataset/URL/spam_dataset.csv", header=None)
l5['result'] = [1 for _ in range(l5.shape[0])]

In [5]:
data = [l1,l2,l3,l4,l5]
dataset = pd.concat(data, ignore_index=True)
dataset = dataset.rename({0: 'url'}, axis=1)
dataset

,url,result
0,http://1337x.to/torrent/1048648/American-Snipe...,0
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0
...,...,...
165361,http://archive.salisburyjournal.co.uk/2001/3/7/,1
165362,http://astore.amazon.co.uk/allezvinsfrenchr/de...,1
165363,http://archive.thisischeshire.co.uk/2000/1/14/...,1
165364,http://applerugs.co.uk/rugs/product_info.php?p...,1


In [6]:
#print sparse matrix
vectorizer = TfidfVectorizer(analyzer='char')
x = vectorizer.fit_transform(dataset['url'])
x.shape

(165366, 133)

In [8]:
ex = x[1].todense()
ex

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.55449616, 0.03830023, 0.2298014 ,
         0.28746351, 0.43139997, 0.20048326, 0.14939271, 0.07685257,
         0.15233965, 0.08369383, 0.17160361, 0.08361285, 0.        ,
         0.0383496 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.11888573, 0.11681316, 0.04138658, 0.13321916,
         0.077861  , 0.06438223, 0.055358  , 0.11494101, 0.03991861,
         0.        , 0.06729052, 0.0864036 , 0.        , 0.08267231,
         0.07915076, 0.07661251, 0.        , 0.17137979, 0.08822504,
         0.26816973, 0.1040908 , 0.        , 0.05091395, 0.06482223,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [11]:
print([(vectorizer.vocabulary_[w],w) for w in vectorizer.get_feature_names()])

[(0, ' '), (1, '!'), (2, '#'), (3, '$'), (4, '%'), (5, '&'), (6, "'"), (7, '('), (8, ')'), (9, '*'), (10, '+'), (11, ','), (12, '-'), (13, '.'), (14, '/'), (15, '0'), (16, '1'), (17, '2'), (18, '3'), (19, '4'), (20, '5'), (21, '6'), (22, '7'), (23, '8'), (24, '9'), (25, ':'), (26, ';'), (27, '<'), (28, '='), (29, '>'), (30, '?'), (31, '@'), (32, '['), (33, '\\'), (34, ']'), (35, '_'), (36, 'a'), (37, 'b'), (38, 'c'), (39, 'd'), (40, 'e'), (41, 'f'), (42, 'g'), (43, 'h'), (44, 'i'), (45, 'j'), (46, 'k'), (47, 'l'), (48, 'm'), (49, 'n'), (50, 'o'), (51, 'p'), (52, 'q'), (53, 'r'), (54, 's'), (55, 't'), (56, 'u'), (57, 'v'), (58, 'w'), (59, 'x'), (60, 'y'), (61, 'z'), (62, '{'), (63, '|'), (64, '}'), (65, '~'), (66, '\x8d'), (67, '\xa0'), (68, '¸'), (69, 'ä'), (70, 'å'), (71, 'ø'), (72, 'š'), (73, 'ˆ'), (74, 'ก'), (75, 'ข'), (76, 'ฃ'), (77, 'ค'), (78, 'ฅ'), (79, 'ง'), (80, 'จ'), (81, 'ฉ'), (82, 'ช'), (83, 'ซ'), (84, 'ฌ'), (85, 'ญ'), (86, 'ฎ'), (87, 'ฏ'), (88, 'ฐ'), (89, 'ฑ'), (90, 'ฒ'), (

In [9]:
#find freq

In [31]:
vectorizer1 = CountVectorizer(analyzer='char')
x1 = vectorizer1.fit_transform(dataset['url'])
x1.shape
np.asarray(x1.sum(axis=0))

array([[     19,      65,      30,      12,  299075,  208609,     293,
            289,     291,     409,   14737,    1578,  414402,  452204,
         751444,  194969,  208990,  186323,  144015,  125967,  138974,
          94753,  101373,  171079,  145291,  190464,    2370,       2,
         292463,       2,   79816,     273,    1486,    3312,    1485,
          91794,  756608,  268647,  577099,  511531, 1048402,  174246,
         226265,  477719,  794977,   70730,  147236,  471149,  463937,
         667389,  799986,  632726,   36739,  537406,  490810, 1125459,
         260502,  154878,  369244,  116770,  140091,   78304,      11,
            216,      11,     312,       1,      25,       1,       1,
              1,       1,       1,       1,      74,      11,      76,
             31,      17,      36,      12,      39,      23,      11,
              3,      38,       8,       6,     330,     159,     214,
            123,      75,     568,      39,      56,     359,      79,
      

In [8]:
vectorizer.vocabulary_

{'h': 43,
 't': 55,
 'p': 51,
 ':': 25,
 '/': 14,
 '1': 16,
 '3': 18,
 '7': 22,
 'x': 59,
 '.': 13,
 'o': 50,
 'r': 53,
 'e': 40,
 'n': 49,
 '0': 15,
 '4': 19,
 '8': 23,
 '6': 21,
 'a': 36,
 'm': 48,
 'i': 44,
 'c': 38,
 '-': 12,
 's': 54,
 '2': 17,
 'd': 39,
 'l': 47,
 'v': 57,
 'b': 37,
 'k': 46,
 '5': 20,
 'u': 56,
 'w': 58,
 'f': 41,
 'g': 42,
 '9': 24,
 'y': 60,
 'q': 52,
 '?': 30,
 '=': 28,
 '_': 35,
 '&': 5,
 '%': 4,
 'z': 61,
 'j': 45,
 '+': 10,
 '(': 7,
 ')': 8,
 '[': 32,
 ']': 34,
 '@': 31,
 '!': 1,
 '|': 63,
 '*': 9,
 ';': 26,
 '{': 62,
 '}': 64,
 "'": 6,
 '~': 65,
 '#': 2,
 '\\': 33,
 'ส': 111,
 'น': 97,
 'ห': 112,
 'พ': 102,
 'ึ': 122,
 'ณ': 91,
 'ว': 109,
 'ล': 107,
 'ต': 93,
 'ฤ': 106,
 'ถ': 94,
 'ฎ': 86,
 'ฮ': 115,
 'บ': 98,
 'ศ': 110,
 'ฅ': 78,
 'ป': 99,
 'ุ': 124,
 'ฟ': 103,
 'ฝ': 101,
 'ฺ': 126,
 'ฐ': 88,
 'ด': 92,
 'ฦ': 108,
 'ท': 95,
 'ๅ': 129,
 'ฉ': 81,
 'ญ': 85,
 'ผ': 100,
 'ู': 125,
 'า': 118,
 'ะ': 116,
 '\uf1dc': 132,
 'ฬ': 113,
 'จ': 80,
 'อ': 114,
 'ไ': 128,

In [90]:
#parameter tuning
lr = RandomForestClassifier()

In [91]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score),
           'log_loss': make_scorer(log_loss)}

In [92]:
#stratified kfold, 
#use train, valid and test(use test_train_split twice)
#log loss across epochs
#SVD(Latent semantic indexing)/PCA
scores = cross_validate(lr, x, dataset['result'], cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([26.85115433, 23.60728049, 28.45226002, 28.94476295, 20.6218276 ]),
 'score_time': array([1.77796435, 1.69069886, 1.92781854, 1.75776076, 1.78316617]),
 'test_accuracy': array([0.94273447, 0.95996734, 0.97562967, 0.95573428, 0.83158468]),
 'train_accuracy': array([0.98529767, 0.98525243, 0.98398252, 0.9850181 , 0.98480645]),
 'test_precision': array([0.94247008, 0.97888281, 0.98992571, 0.99205776, 0.96482206]),
 'train_precision': array([0.99192063, 0.99148428, 0.99074128, 0.99127336, 0.99190608]),
 'test_recall': array([0.98742211, 0.96999654, 0.97895911, 0.95130395, 0.81548581]),
 'train_recall': array([0.98935475, 0.9897395 , 0.98886442, 0.98965285, 0.9887393 ]),
 'test_f1': array([0.96442257, 0.97441941, 0.98441187, 0.97125353, 0.8838906 ]),
 'train_f1': array([0.99063603, 0.99061112, 0.98980196, 0.99046244, 0.99032016]),
 'test_log_loss': array([1.97791907, 1.38269207, 0.84172772, 1.52888843, 5.81687789]),
 'train_log_loss': array([0.50780539, 0.50936829, 0.5532

In [93]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Train Log Loss is: ",scores['train_log_loss'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")
print("The Test Log Loss is: ",scores['test_log_loss'].mean())
print("\n")

The Fit Time is:  25.69545707702637


The Score Time is:  1.7874817371368408


The Train Accuracy score is:  0.984871437336329


The Train F1 score is:  0.9903663409879344


The Train Precision score is:  0.9914651242756338


The Train Recall score is:  0.9892701632773966


The Train Log Loss is:  0.5225273957150853


The Test Accuracy score is:  0.933130089592875


The Test F1 score is:  0.9556795977928674


The Test Precision score is:  0.973631683406459


The Test Recall score is:  0.9406335022359315


The Test Log Loss is:  2.3096210377701603




## Multi-Class Classification

In [3]:
#check 5 class as well
l2=pd.read_csv("FinalDataset/URL/DefacementSitesURLFiltered.csv", header=None)
l2['result'] = [1 for _ in range(l2.shape[0])]
l2['type'] = 'Defacement'
l2

,0,result,type
0,http://www.sinduscongoias.com.br/index.html,1,Defacement
1,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
2,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
3,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
4,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
...,...,...,...
96452,http://www.laurenzano.com.ar/index.php?option=...,1,Defacement
96453,http://www.laurenzano.com.ar/templates/laurenz...,1,Defacement
96454,http://www.laurenzano.com.ar/templates/laurenz...,1,Defacement
96455,http://www.laurenzano.com.ar/x.txt,1,Defacement


In [4]:
l3=pd.read_csv("FinalDataset/URL/Malware_dataset.csv", header=None)
l3['result'] = [2 for _ in range(l3.shape[0])]
l3['type'] = 'Malware'
l3

,0,result,type
0,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
1,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
2,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
3,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,2,Malware
...,...,...,...
11561,http://www.plastischechirurgie.net/gesicht/fac...,2,Malware
11562,http://www.plastischechirurgie.net/haartranspl...,2,Malware
11563,http://www.plastischechirurgie.net/korper/gyna...,2,Malware
11564,http://dl.get1993desk.com/n/50517366/RealPlaye...,2,Malware


In [5]:
l4=pd.read_csv("FinalDataset/URL/phishing_dataset.csv", header=None)
l4['result'] = [3 for _ in range(l4.shape[0])]
l4['type'] = 'Phishing'
l4

,0,result,type
0,http://v2.email-marketing.adminsimple.com/trac...,3,Phishing
1,http://bid.openx.net/json?amp;amp;amp;amp;cid;...,3,Phishing
2,http://webmail2.centurytel.net/hwebmail/servic...,3,Phishing
3,http://www.google.com.ng/imgres?imgurl=http://...,3,Phishing
4,http://webmail2.centurytel.net/hwebmail/servic...,3,Phishing
...,...,...,...
9960,http://highedgesolar.com/nw/includes/,3,Phishing
9961,http://jimfangimporters.yolasite.com/,3,Phishing
9962,http://liuheng.chengxuren.com/Images/,3,Phishing
9963,http://mcnaotempreco.net/site/portal/,3,Phishing


In [6]:
l5=pd.read_csv("FinalDataset/URL/spam_dataset.csv", header=None)
l5['result'] = [4 for _ in range(l5.shape[0])]
l5['type'] = 'Spam'
l5

,0,result,type
0,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
1,http://archive.salisburyjournal.co.uk/2007/3/6...,4,Spam
2,http://appbasic.jettons.co.uk/links/index.html,4,Spam
3,http://archive.yorkpress.co.uk/2003/11/6/25684...,4,Spam
4,http://acard4u.co.uk/product_reviews.php?cPath...,4,Spam
...,...,...,...
11995,http://archive.salisburyjournal.co.uk/2001/3/7/,4,Spam
11996,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
11997,http://archive.thisischeshire.co.uk/2000/1/14/...,4,Spam
11998,http://applerugs.co.uk/rugs/product_info.php?p...,4,Spam


In [7]:
data=[l2,l3,l4,l5]
urls = pd.concat(data)
urls = urls.rename({0: 'url'}, axis=1)
urls
urls

,url,result,type
0,http://www.sinduscongoias.com.br/index.html,1,Defacement
1,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
2,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
3,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
4,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
...,...,...,...
11995,http://archive.salisburyjournal.co.uk/2001/3/7/,4,Spam
11996,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
11997,http://archive.thisischeshire.co.uk/2000/1/14/...,4,Spam
11998,http://applerugs.co.uk/rugs/product_info.php?p...,4,Spam


In [8]:
vectorizer = TfidfVectorizer(analyzer='char')
x = vectorizer.fit_transform(urls['url'])
x.shape

(129988, 133)

In [9]:
#lr = RandomForestClassifier()
clf = SVC()

In [10]:
#metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')}

In [11]:
scores = cross_validate(clf, x, np.array(urls['result'].tolist()), cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([32.53318429, 31.62723327, 32.1542058 , 31.35656047, 24.68078446]),
 'score_time': array([3.19092965, 3.06915045, 2.72167206, 2.41347694, 2.34113979]),
 'test_accuracy': array([0.92303254, 0.95291946, 0.94172629, 0.92526061, 0.88991037]),
 'train_accuracy': array([0.96513126, 0.96501587, 0.96838158, 0.97126674, 0.97065131]),
 'test_precision': array([0.92252203, 0.95178214, 0.94068401, 0.9212951 , 0.86922313]),
 'train_precision': array([0.96487962, 0.96477056, 0.96830244, 0.97116002, 0.97082653]),
 'test_recall': array([0.92303254, 0.95291946, 0.94172629, 0.92526061, 0.88991037]),
 'train_recall': array([0.96513126, 0.96501587, 0.96838158, 0.97126674, 0.97065131]),
 'test_f1': array([0.92229495, 0.95129652, 0.93775406, 0.91453332, 0.8699475 ]),
 'train_f1': array([0.96327495, 0.96317291, 0.96685817, 0.97017557, 0.9693001 ])}

In [12]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")

The Fit Time is:  30.470393657684326


The Score Time is:  2.7472737789154054


The Train Accuracy score is:  0.9680893510567351


The Train F1 score is:  0.9665563418972114


The Train Precision score is:  0.9679878310974648


The Train Recall score is:  0.9680893510567351


The Test Accuracy score is:  0.9265698528423822


The Test F1 score is:  0.9191652703791806


The Test Precision score is:  0.9211012822647527


The Test Recall score is:  0.9265698528423822




## 5-Class Multi-Class Classification

In [3]:
l1=pd.read_csv("FinalDataset/URL/Benign_list_big_final.csv", header=None)
l1['result'] = [0 for _ in range(l1.shape[0])]
l1['type'] = 'Benign'
l1

,0,result,type
0,http://1337x.to/torrent/1048648/American-Snipe...,0,Benign
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0,Benign
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0,Benign
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0,Benign
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0,Benign
...,...,...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...,0,Benign
35374,https://lastpass.com/signup2.php?ac=1&from_uri...,0,Benign
35375,https://lastpass.com/signup2.php?ac=1&from_uri...,0,Benign
35376,https://lastpass.com/signup2.php?ac=1&from_uri...,0,Benign


In [4]:
l2=pd.read_csv("FinalDataset/URL/DefacementSitesURLFiltered.csv", header=None)
l2['result'] = [1 for _ in range(l2.shape[0])]
l2['type'] = 'Defacement'
l2

,0,result,type
0,http://www.sinduscongoias.com.br/index.html,1,Defacement
1,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
2,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
3,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
4,http://www.sinduscongoias.com.br/index.php/ins...,1,Defacement
...,...,...,...
96452,http://www.laurenzano.com.ar/index.php?option=...,1,Defacement
96453,http://www.laurenzano.com.ar/templates/laurenz...,1,Defacement
96454,http://www.laurenzano.com.ar/templates/laurenz...,1,Defacement
96455,http://www.laurenzano.com.ar/x.txt,1,Defacement


In [5]:
l3=pd.read_csv("FinalDataset/URL/Malware_dataset.csv", header=None)
l3['result'] = [2 for _ in range(l3.shape[0])]
l3['type'] = 'Malware'
l3

,0,result,type
0,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
1,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
2,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
3,http://gzzax.livechatvalue.com/chat/chatClient...,2,Malware
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,2,Malware
...,...,...,...
11561,http://www.plastischechirurgie.net/gesicht/fac...,2,Malware
11562,http://www.plastischechirurgie.net/haartranspl...,2,Malware
11563,http://www.plastischechirurgie.net/korper/gyna...,2,Malware
11564,http://dl.get1993desk.com/n/50517366/RealPlaye...,2,Malware


In [6]:
l4=pd.read_csv("FinalDataset/URL/phishing_dataset.csv", header=None)
l4['result'] = [3 for _ in range(l4.shape[0])]
l4['type'] = 'Phishing'
l4

,0,result,type
0,http://v2.email-marketing.adminsimple.com/trac...,3,Phishing
1,http://bid.openx.net/json?amp;amp;amp;amp;cid;...,3,Phishing
2,http://webmail2.centurytel.net/hwebmail/servic...,3,Phishing
3,http://www.google.com.ng/imgres?imgurl=http://...,3,Phishing
4,http://webmail2.centurytel.net/hwebmail/servic...,3,Phishing
...,...,...,...
9960,http://highedgesolar.com/nw/includes/,3,Phishing
9961,http://jimfangimporters.yolasite.com/,3,Phishing
9962,http://liuheng.chengxuren.com/Images/,3,Phishing
9963,http://mcnaotempreco.net/site/portal/,3,Phishing


In [7]:
l5=pd.read_csv("FinalDataset/URL/spam_dataset.csv", header=None)
l5['result'] = [4 for _ in range(l5.shape[0])]
l5['type'] = 'Spam'
l5

,0,result,type
0,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
1,http://archive.salisburyjournal.co.uk/2007/3/6...,4,Spam
2,http://appbasic.jettons.co.uk/links/index.html,4,Spam
3,http://archive.yorkpress.co.uk/2003/11/6/25684...,4,Spam
4,http://acard4u.co.uk/product_reviews.php?cPath...,4,Spam
...,...,...,...
11995,http://archive.salisburyjournal.co.uk/2001/3/7/,4,Spam
11996,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
11997,http://archive.thisischeshire.co.uk/2000/1/14/...,4,Spam
11998,http://applerugs.co.uk/rugs/product_info.php?p...,4,Spam


In [8]:
data=[l1,l2,l3,l4,l5]
urls = pd.concat(data)
urls = urls.rename({0: 'url'}, axis=1)
urls
urls

,url,result,type
0,http://1337x.to/torrent/1048648/American-Snipe...,0,Benign
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0,Benign
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0,Benign
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0,Benign
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0,Benign
...,...,...,...
11995,http://archive.salisburyjournal.co.uk/2001/3/7/,4,Spam
11996,http://astore.amazon.co.uk/allezvinsfrenchr/de...,4,Spam
11997,http://archive.thisischeshire.co.uk/2000/1/14/...,4,Spam
11998,http://applerugs.co.uk/rugs/product_info.php?p...,4,Spam


In [36]:
vectorizer = CountVectorizer(analyzer='char')
x = vectorizer.fit_transform(urls['url'])
x.shape

(165366, 133)

In [57]:
#LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,SVC
clf = LogisticRegression()

In [58]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')}

In [59]:
scores = cross_validate(clf, x, np.array(urls['result'].tolist()), cv=5,scoring=scoring,return_train_score=True, verbose = 3)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END  accuracy: (train=0.865, test=0.834) f1: (train=0.857, test=0.826) precision: (train=0.857, test=0.826) recall: (train=0.865, test=0.834) total time=   8.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s remaining:    0.0s


[CV] END  accuracy: (train=0.868, test=0.849) f1: (train=0.860, test=0.845) precision: (train=0.860, test=0.843) recall: (train=0.868, test=0.849) total time=   7.5s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.9s remaining:    0.0s


[CV] END  accuracy: (train=0.868, test=0.853) f1: (train=0.860, test=0.847) precision: (train=0.860, test=0.848) recall: (train=0.868, test=0.853) total time=   7.8s
[CV] END  accuracy: (train=0.865, test=0.838) f1: (train=0.859, test=0.819) precision: (train=0.860, test=0.815) recall: (train=0.865, test=0.838) total time=   7.8s
[CV] END  accuracy: (train=0.880, test=0.785) f1: (train=0.875, test=0.761) precision: (train=0.876, test=0.753) recall: (train=0.880, test=0.785) total time=   7.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   39.7s finished


{'fit_time': array([8.05825377, 7.57315826, 7.82130408, 7.8115778 , 7.6894846 ]),
 'score_time': array([0.04787207, 0.04595661, 0.04491186, 0.04242468, 0.04496384]),
 'test_accuracy': array([0.833676  , 0.84881928, 0.85262903, 0.83775285, 0.78523267]),
 'train_accuracy': array([0.86486711, 0.86759693, 0.86775566, 0.86545017, 0.8802431 ]),
 'test_precision': array([0.82554461, 0.84260761, 0.84755107, 0.81470398, 0.75340205]),
 'train_precision': array([0.85705155, 0.86005668, 0.86048788, 0.85973296, 0.87578488]),
 'test_recall': array([0.833676  , 0.84881928, 0.85262903, 0.83775285, 0.78523267]),
 'train_recall': array([0.86486711, 0.86759693, 0.86775566, 0.86545017, 0.8802431 ]),
 'test_f1': array([0.82633672, 0.84452684, 0.84704734, 0.81870959, 0.76145616]),
 'train_f1': array([0.85707574, 0.86004345, 0.85993898, 0.85892435, 0.87487726])}

In [60]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")

The Fit Time is:  7.790755701065064


The Score Time is:  0.04522581100463867


The Train Accuracy score is:  0.8691825944942092


The Train F1 score is:  0.8621719553519551


The Train Precision score is:  0.8626227913123291


The Train Recall score is:  0.8691825944942092


The Test Accuracy score is:  0.8316219654945165


The Test F1 score is:  0.8196153301777243


The Test Precision score is:  0.8167618650353548


The Test Recall score is:  0.8316219654945165




## SVD

In [14]:
from sklearn.decomposition import TruncatedSVD

In [15]:
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)

In [16]:
x_new = svd.fit_transform(x)

In [17]:
#check add -ve values in alexander paper 30-40 pages
x_new[0]

array([ 7.28421687e-01,  3.74607797e-01, -1.15225801e-02,  2.35349467e-01,
        2.26210879e-01,  1.23806330e-01, -1.20576654e-01,  1.12070354e-01,
        5.38868221e-02, -4.97951195e-03,  8.62243711e-02, -2.41305346e-02,
        2.30548204e-02, -6.34655357e-02,  2.85524587e-03, -3.93702817e-02,
        3.62505267e-02,  1.45274889e-01,  5.65417065e-03, -7.85956377e-02,
       -5.64743954e-02, -2.99128663e-02,  2.60937625e-02, -8.85200669e-02,
        2.05157438e-02,  7.15887701e-02,  9.09113485e-03,  8.93406460e-02,
       -6.50631072e-02, -1.55681791e-02,  2.18187277e-02, -5.20529924e-02,
        5.94365533e-02,  1.53667069e-01,  4.74190135e-02,  1.85318284e-01,
        6.23526896e-02, -8.13638636e-02,  7.99782425e-02,  2.60667271e-02,
       -7.44375783e-02,  1.06770988e-01, -3.40286109e-02,  1.88465180e-02,
       -2.26420578e-02, -4.19640188e-04,  4.15645674e-03, -7.92197255e-04,
       -5.82325284e-03, -1.84778194e-03,  3.07447770e-03, -4.96173953e-03,
       -2.44999607e-04, -

In [18]:
clf = SVC()

In [19]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')}

In [20]:
scores = cross_validate(clf, x_new, np.array(urls['result'].tolist()), cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([23.602988  , 21.87445664, 21.45664334, 23.47521186, 24.67899156]),
 'score_time': array([2.15418887, 2.47338462, 2.34968352, 2.25297308, 2.63993859]),
 'test_accuracy': array([0.87751708, 0.9339945 , 0.92356303, 0.91990445, 0.86847277]),
 'train_accuracy': array([0.95856892, 0.95612013, 0.9592571 , 0.9606026 , 0.9595519 ]),
 'test_precision': array([0.88367977, 0.93342809, 0.92362194, 0.91598886, 0.84615493]),
 'train_precision': array([0.95827406, 0.95585926, 0.95920051, 0.96055764, 0.95957714]),
 'test_recall': array([0.87751708, 0.9339945 , 0.92356303, 0.91990445, 0.86847277]),
 'train_recall': array([0.95856892, 0.95612013, 0.9592571 , 0.9606026 , 0.9595519 ]),
 'test_f1': array([0.87924866, 0.93245056, 0.92029938, 0.91066959, 0.85288389]),
 'train_f1': array([0.95688173, 0.95438509, 0.95780722, 0.95955205, 0.95827737])}

In [21]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")

The Fit Time is:  23.017658281326295


The Score Time is:  2.374033737182617


The Train Accuracy score is:  0.9588201319327905


The Train F1 score is:  0.9573806906547017


The Train Precision score is:  0.9586937230784693


The Train Recall score is:  0.9588201319327905


The Test Accuracy score is:  0.9046903666611261


The Test F1 score is:  0.8991104151218845


The Test Precision score is:  0.9005747183462361


The Test Recall score is:  0.9046903666611261


